In [ ]:
## Data Preparation
# List of functions for loading German Traffic Sign Data Set for (Training and Testing)

In [7]:
## Import libraries need to be imported, Dont forget to update requirements.txt!
import os
import sys
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 

In [8]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(_dir):
    train_category_dir= os.path.join(training_dataset_dir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path)
    return train_category_dir, train_Data_frame

In [9]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [10]:
## Read image in path
def readImage(path):
    return cv2.imread(path)

In [13]:
def loadSampleImg():
    img=readImage('../DataSet/Training_DataSet/Final_Training/Images/00000/00000_00000.ppm')
    cv2.imshow('image',img)
    cv2.waitKey()

In [ ]:
## Training DataSet Directory
training_dataset_dir = '../DataSet/Training_DataSet/Final_Training/Images'
for _dir in os.listdir(training_dataset_dir):
    train_category_dir, train_Data_frame= getDirAndDataFrame(_dir)
    
    for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):
        img=readImage(img_path)